# **Maestría en Inteligencia Artificial Aplicada**
## **Curso: Inteligencia Artificial y Aprendizaje Automático**
### Tecnológico de Monterrey
### Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 7**
### **Red Neuronal Artificial - Perceptrón Multicapa : Multilayer Perceptrón (MLP)**


**Nombres y matrículas de los integrantes del equipo:**

*   Carlos Enríquez Gorgonio A01793102
*   Fernando Ramírez Gómez A01298109
*   Francisco Javier Ramírez Arias A01316379
*   Jessica Lechuga Ramos A01793512
*   Moisés Rangel Silva A00143704

Esta Tarea se deberá resolver en equipos, de acuerdo a como fueron integrados al inicio del curso. 

El artículo del 2016 de Moro, Rita & Vala relacionado al presente problema lo encuentras aquí: https://www.semanticscholar.org/paper/Predicting-social-media-performance-metrics-and-of-Moro-
Rita/dec55692590820754b53c916e29bb2b42c0e5104  
 
Y el conjunto de datos utilizado es el de la siguiente liga de la UCI: https://archive.ics.uci.edu/ml/datasets/Facebook+metrics 


El archivo se llama dataset_Facebook.csv y contiene 500 registros y 19 columnas.  El objetivo es determinar el impacto generado por un anuncio de una marca de cosméticos en  Facebook, en el cual se intentaron varias variantes en la manera de mostrar el anuncio. Siguiendo el  artículo, deberás considerar como predictores o variables de entrada aquellas que se indican en la Tabla 3. Por otro lado, en la Tabla 2, los autores consideran varios casos para la variable de salida, intentando  determinar cuál puede ser el mejor caso para medir el éxito de la campaña. Para este ejercicio deberás  considerar únicamente los siguientes tres casos como variable de salida: “Lifetime post consumers”,  “Lifetime People who have liked a Page and engaged with a post” y “Likes”.

En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [1]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.

import graphviz
import math
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import pydotplus
import seaborn as sns

from IPython.display import Image
from sklearn import tree
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, train_test_split, cross_validate, learning_curve
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

# **Ejercicio-1.** 

En esta tarea considera únicamente la siguiente variable de salida que se concluye que es una de las 
mejores en el artículo antes citado:  ‘Lifetime People who have liked a Page and engaged with a 
post'. Renombra dicha variable como “LPE” . Como variables de entrada selecciona las 7 variables 
que indican los autores en la Tabla 3 del artículo citado. 

In [15]:
#Importación de archivo a DataFrame
#df= pd.read_csv('https://raw.githubusercontent.com/PosgradoMNA/actividades-de-aprendizaje-KarltonBotics/main/DataFrames/SouthGermanCredit.asc', sep='\s+')
df= pd.read_csv('dataset_Facebook.csv', sep=';')
df.head()

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


In [16]:
#Sustituye los nombres de las columnas del alemán al inglés de acuerdo a como se indica en la página de la UCI.
df.rename(columns = {
    'Page total likes': 'total_likes',
    'Type': 'type',
    'Category': 'category',
    'Post Month': 'post_month',
    'Post Weekday': 'post_weekday',
    'Post Hour': 'post_hour',
    'Paid': 'paid',
    'Lifetime Post Total Reach': 'lptr',
    'Lifetime Post Total Impressions': 'lpti',
    'Lifetime Engaged Users': 'leu',
    'Lifetime Post Consumers': 'lpcm',
    'Lifetime Post Consumptions': 'lpcn',
    'Lifetime Post Impressions by people who have liked your Page': 'lpiplp',
    'Lifetime Post reach by people who like your Page': 'lprplp',
    'Lifetime People who have liked your Page and engaged with your post': 'lphlp',
    'comment': 'comment',
    'like': 'like',
    'share': 'share',
    'Total Interactions': 'interactions'
}, inplace = True)
#El inplace true, nos permite hacer la mutación sobre el mismo dataframe, sin necesidad de crear otro nuevo
df.head()

,total_likes,type,category,post_month,post_weekday,post_hour,paid,lptr,lpti,leu,lpcm,lpcn,lpiplp,lprplp,lphlp,comment,like,share,interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


In [19]:
#Determinamos las columnas que tenga el eje x
X = df[['category', 'total_likes', 'type', 'post_month', 'post_hour', 'post_hour', 'paid']].copy()
print("Columnas X")
print(X)
#Determinamos las columnas que tenga el eje y
y = df[['lpcm', 'lphlp', 'like']].copy()
print("\nColumnas y")
print(y)

Columnas X
     category  total_likes    type  post_month  post_hour  post_hour  paid
0           2       139441   Photo          12          3          3   0.0
1           2       139441  Status          12         10         10   0.0
2           3       139441   Photo          12          3          3   0.0
3           2       139441   Photo          12         10         10   1.0
4           2       139441   Photo          12          3          3   0.0
..        ...          ...     ...         ...        ...        ...   ...
495         3        85093   Photo           1          2          2   0.0
496         2        81370   Photo           1          8          8   0.0
497         1        81370   Photo           1          2          2   0.0
498         3        81370   Photo           1         11         11   0.0
499         2        81370   Photo           1          4          4   NaN

[500 rows x 7 columns]
Columnas y
     lpcm  lphlp    like
0     109    119    79.0
1   

# **Ejercicio-2.**

Realiza una partición de los datos con 100 datos de Prueba y el resto para entrenamiento y 
validación.  

In [24]:
#creamos nuestros datos de validación con la función trains_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, random_state=10)

print("Tamaño del conjunto de entrenamiento y validación: ", X_train.shape)
print("Tamaño del conjunto de prueba: ", X_test.shape)

Tamaño del conjunto de entrenamiento y validación:  (400, 7)
Tamaño del conjunto de prueba:  (100, 7)


# **Ejercicio-3.**

Definirás tus propias funciones de errores para este problema de regresión. Los errores que 
utilizarás son la raíz cuadrada del error cuadrático medio RMSE, el error absoluto medio MAE y el 
error porcentual absoluto medio MAPE. 

In [ ]:
#Raíz del error cuadrático medio
def rmse(y, y_hat):
    substraction = (y - y_hat)
    square = np.square(substraction)
    MSE = square.mean()
    RMSE = np.sqrt(MSE)

    return RMSE

In [ ]:
#Error absoluto medio
def mae(y, y_hat):
    absolute = np.abs(y - y_hat)
    avarage = np.mean(absolute)

    return avarage

In [ ]:
#Error absoluto medio porcentual
def mape(y, y_hat):
    absolute = np.abs((y - y_hat) / y)
    avarage = np.mean(absolute)
    percentage = avarage * 100

    return percentage

# **Ejercicio-4.**

En la página de la UCI, así como en el artículo de los autores previamente citado encuentras información en relación al significado de cada variable. Haz una análisis de tus datos y lleva a cabo las transformaciones que consideres adecuadas tanto en los datos de entrada, como en las de salida. Utiliza un Pipeline para evitar el filtrado de información. 

In [ ]:
None

# **Ejercicio-5.**

Utiliza la función Dummy para modelos de regresión de scikit-learn con el conjunto que tienes de datos de entrenamiento y validación. Para ello particiónalos en 100 para validación y 300 para  entrenamiento. Encuentra los errores RMSE, MAE y MAPE para los conjuntos de entrenamiento y  validación. Estos serán tus errores máximos que deberás tomar como referencia en el resto de la  actividad. Consulta su documentación correspondiente: https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyRegressor.html

In [ ]:
None

# **Ejercicio-6.**

Usando los modelos de regresión lineal múltiple, el bosque aleatorio y el perceptrón multicapa con  sus valores predeterminados, lleva a cabo su entrenamiento con repeticiones de validación cruzada (RepeatedKFold) y desplegando los errores RMSE, MAE y MAPE. Recuerda evitar el filtrado de información usando los datos que obtuviste en el ejercicio 2. Incluye las conclusiones sobre el mejor modelo encontrado en esta primera aproximación. En particular ¿hay alguno sobreentrenado o subentrenado? NOTA: Recuerda que puedes aumentar en dado caso el número máximo de iteraciones para que todos los modelos converjan.

In [ ]:
None

# **Ejercicio-7.**

Obtener los diagramas de caja y bigote para los errores MAPE de los conjuntos de validación obtenidos. En particular compara estos primeros resultados de MAPE con el mejor resultado que  encuentran los autores del artículo citado al inicio. Incluye tus conclusiones.

In [ ]:
None

# **Ejercicio-8.**

Usando una búsqueda de malla con validación cruzada (GridSearchCV), busca los mejores hiperparámetros para el modelo MLP. Al menos deberás realizar la búsqueda en los  hiperparámetros “hidden_layer_sizes”, “alpha” y “learning_rate_init”. Además aplica la validación  cruzada con repeticiones (RepeatedKFold). Muestra los mejores hiperparámetros encontrados. 

In [ ]:
None

# **Ejercicio-9.**

Con los mejores valores de los hiperparámetros encontrados realiza un análisis de la importancia de los factores. Muestra un diagrama de barras de los resultados e incluye tus conclusiones.

In [ ]:
None

# **Ejercicio-10.**

Repite el ejercicio 8 y 9 para el modelo de bosque aleatorio para buscar sus mejores hiperparámetros (realiza la búsqueda con aquellos hiperparámetros que consideres más adecuados) y usando el conjunto de Prueba. Y realiza igualmente el análisis de importancia de  factores con este modelo con un diagrama de barras. 

In [ ]:
None

# **Ejercicio-11.**

Repite el ejercicio 8 y 9 para el modelo de regresión lineal múltiple para buscar sus mejores hiperparámetros (realiza la búsqueda con aquellos hiperparámetros que consideres más adecuados) y usando el conjunto de Prueba. Y realiza igualmente el análisis de importancia de factores con este modelo con un diagrama de barras. 

In [ ]:
None

# **Ejercicio-12.**

Compara tus resultados con los obtenidos por los autores del artículo de Moro-Rita-Vala con respecto a MAPE. Incluye tus conclusiones finales de la actividad.

In [ ]:
None

### **Fin de la Actividad de la semana 7.**